# Training a ConvNet with the ImageDataGenerator from TF

In [1]:
!nvidia-smi

Tue Jun 27 18:29:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti      On | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P3               16W /  30W|    763MiB /  4096MiB |     30%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array, plot_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Parameters for our graph; we'll output images in a 4x4 configuration
nRows = 4
nCols = 4

# Index for iterating over images
pic_index = 0

2023-06-27 18:29:04.612365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-27 18:29:04.612387: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Check if TensorFlow is using GPU
print("GPU Available:", tf.test.is_gpu_available())

# Get information about the available GPUs
print("GPU Devices:")
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available: False
GPU Devices:
[]


2023-06-27 18:29:07.229321: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 18:29:07.249831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-27 18:29:07.250145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-27 18:29:07.250224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such 

## Download and Unzip Data

In [4]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip --directory-prefix {'./assets/data'}

--2023-06-27 18:29:07--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:4019:80c::2010, 2a00:1450:4019:80d::2010, 2a00:1450:4019:80a::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:4019:80c::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘./assets/data/horse-or-human.zip.1’

horse-or-human.zip.  51%[=========>          ]  73.77M  17.9MB/s    eta 6s     ^C


In [ ]:
import zipfile

# Unzip the dataset
local_zip = './assets/data/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./assets/data/horse-or-human')

zip_ref.close()

## Load Images

In [ ]:
import os

# Directory with our training horse pictures
horseTrainingDirectory = os.path.join('./assets/data/horse-or-human/horses')

# Directory with our training human pictures
humanTrainingDirectory = os.path.join('./assets/data/horse-or-human/humans')

In [ ]:
horseTrainingNames = os.listdir(horseTrainingDirectory)
print(horseTrainingNames[:10])

humanTrainingNames = os.listdir(humanTrainingDirectory)
print(humanTrainingNames[:10])

In [ ]:
print('Total training horse images:', len(os.listdir(horseTrainingDirectory)))
print('Total training human images:', len(os.listdir(humanTrainingDirectory)))

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()

fig.set_size_inches(nCols * 4, nRows * 4)

pic_index += 8

next_horse_pix = [os.path.join(horseTrainingDirectory, fname) for fname in horseTrainingNames[pic_index-8:pic_index]]

next_human_pix = [os.path.join(humanTrainingDirectory, fname) for fname in humanTrainingNames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nRows, nCols, i + 1)
    
    sp.axis('Off') # Don't show axes (or grid lines)

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

## Create Model

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## Visualise Model

In [ ]:
print(model.summary())

In [ ]:
plot_model(model, to_file='./assets/model/model.png', show_shapes=True, show_layer_names=True)

## Compile Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
# All images will be rescaled by 1./255
trainingImageGenerator = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using trainingGenerator generator
trainGenerator = trainingImageGenerator.flow_from_directory(
	'./assets/data/horse-or-human/',  # This is the source directory for training images
	target_size=(300, 300),  # All images will be resized to 300x300
	batch_size=128,
	# Since we use binary_crossentropy loss, we need binary labels
	class_mode='binary'
)

## Train Model

In [ ]:
history = model.fit(
      trainGenerator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1
)

## Predict for a Test Image

In [ ]:
# predicting images
fileName = 'predict.jpg'
path = './assets/data/predict/' + fileName

img = load_img(path, target_size=(300, 300))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])

classes = model.predict(images, batch_size=10)

print(classes[0])

if classes[0] > 0.5:
    print(fileName + " is a human")
else:
    print(fileName + " is a horse")

## Visualise the Convolutions

In [ ]:
import numpy as np
import random
from tensorflow.keras.utils import img_to_array, load_img

# Define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepare a random input image from the training set.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (300, 300, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 300, 300, 3)

# Scale by 1/255
x /= 255

# Run the image through the network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so you can have them as part of the plot
layer_names = [layer.name for layer in model.layers[1:]]

# Display the representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:

    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map

    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    
    # Tile the images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
    
      # Tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

---